In [ ]:
# Imports
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from math import pi
from time import time
import minterpy_levelsets as ls

# Local imports
import surfgeopy as sp

# Gauss Bonnet theorem on ellipsoid
This is a benchmark of computing surface integrals using high-order surface quadrature (HOSQ) method and GPLS method for algebraic varieties.



# Step 1: Load and extract the vertices of a ellipsoid mesh composed of          triangles.

In [ ]:
mesh_path="../meshes/Ellipsoid_N=4024.mat"
# ellipsoid_N=6152_n.mat
mesh_mat = scipy.io.loadmat(mesh_path)
pointcloud= mesh_mat["xs"]

In [ ]:
a=0.6
b=0.8
c=2.0
def phi(x: np.ndarray):
    return (x[0]**2/a**2+x[1]**2/b**2+x[2]**2/c**2)-1


def dphi(x: np.ndarray):
    return np.array([2*x[0]/a**2, 2*x[1]/b**2, 2*x[2]/c**2])

# Step 2: Perform surface fitting

In [ ]:
result = np.zeros(pointcloud.shape[0])  # initialize an empty array to store the results
for i in range(pointcloud.shape[0]):
    result[i] = phi(pointcloud[i])
print(f"The accuracy of the given mesh is:{(result).max()}")
if (result).max()>1e-10:
    for i in range(pointcloud.shape[0]):
        pointcloud[i,:]=sp.SimpleImplicitSurfaceProjection(phi, dphi,pointcloud[i,:])
newt_poly = ls.LevelsetPoly(pointcloud, method='BK',verbose=True)

# Step 3: We execute  HOSQ for the ellipsoid.


This step involves utilizing the zero level set provided by GPLS and computing the curvature:

If $M$ is orienatble, then  the gradient $\nabla Q_M = (\partial_x Q_M,\, \partial_y Q_M,\, \partial_z Q_M) \in \mathbb{R}^3$ never vanishes on $M$ and provides, together with the Hessian $H_M =\nabla(\nabla Q_M) \in \mathbb{R}^{3 \times 3}$ of $Q_M$
\begin{equation*}
 H_M  = \left(\begin{array}{ccc}
 \frac{\partial^2 Q_M}{\partial_{x}^2} & \frac{\partial^2 Q_M}{\partial_{x}\partial_y} & \frac{\partial^2 Q_M}{\partial_{x}\partial_z} \\
 \frac{\partial^2 Q_M}{\partial_{y}\partial_x} & \frac{\partial^2 Q_M}{\partial_y^2} & \frac{\partial^2 Q_M}{\partial_{y}\partial_z} \\
 \frac{\partial^2 Q_M}{\partial_z\partial_{x}} & \frac{\partial^2 Q_M}{\partial_{z}\partial_y} & \frac{\partial^2 Q_M}{\partial_z^2} \\
 \end{array}\right) ,
\end{equation*}
the main ingredients for the following computations. Both Gauss and mean curvature can be computed from these quantities \cite{goldman2005} as:
\begin{align}
  K_{\mathrm{Gauss}} &= \frac{\det \left(\begin{array}{cc}H_M  & \nabla Q_M^T \\  \nabla Q_M & 0 \end{array}\right)}{\|\nabla Q_M\|^4} \label{eq:GC}\\
  K_{\mathrm{mean}} &= \frac{\nabla Q_M H_M \nabla Q_M^T - \|\nabla Q_M\|^2\mathrm{trace}(H_M)}{2\|\nabla Q_M\|^3}\,. \label{eq:MC}
\end{align}

In [ ]:
def err_t(intp_degree,lp_degree,func,grad_fun,mesh_path,Refinement):
    mesh_mat = scipy.io.loadmat(mesh_path)
    xs =mesh_mat["xs"]
    elems = mesh_mat["surfs"] - 1
    t0 = time()
    pnts, ws, offset = sp.compute_surf_quadrature(func,grad_fun, xs, elems,intp_degree,lp_degree,Refinement)
    # number of quadrature points
    nqp = pnts.shape[0]
    #compute the mean and Gauss curvature using GPLS
    mean_curvature, gauss_curvature = newt_poly.compute_curvatures_at(pnts)
    # function value at each quadrature points
    fs_qp = np.array(
            [
                    gauss_curvature[qpid]

                for qpid in range(nqp)
            ]
        )
    # numerical integrations
    nints = np.matmul(ws, fs_qp)
    exact_area =4*np.pi
    t1 = time()
    print("Relative error: ", abs( nints - exact_area)/exact_area)
    print ("The main function takes:",{(t1-t0)})
    error=abs( nints - exact_area)/exact_area
    return error

# here is the error obtained using Dune Curved Grid
error_dune2_14=np.array([3.871663e-06, 2.012749e-06, 2.485905e-09, 6.281589e-10,
                         7.354117e-13, 2.074785e-11, 6.546763e-11, 9.405419e-10,
                         8.085156e-09, 7.633022e-08, 2.006434e-07, 2.934030e-06, 2.848395e-05])

dune2_14_N_4024=np.array([ 2.567782e-04, 7.830836e-05, 5.154306e-06, 8.168375e-07, 9.032239e-08, 7.802413e-09, 1.883381e-09, 5.726655e-10, 8.552442e-09, 5.485195e-08, 6.194242e-07, 1.826205e-06, 1.550784e-05])

eror_HOSQ_GPLS=np.array([3.324983573425345e-07,4.615327494561246e-08,1.9531970064249053e-10,
                         9.223891292215884e-12,6.536393265372134e-13,1.1308638867425838e-14,
                         4.947529504498804e-15,4.523455546970335e-15,4.806171518655981e-15,
                         6.219751377084211e-15,5.2302454761844496e-15,4.947529504498804e-15,5.654319433712919e-15])

eror_HOSQ_GPLS_4024=np.array([8.707592697508246e-06,2.8670754858619657e-06, 6.34381555626003e-08,
                              2.2520193070174823e-09,8.538927036015901e-11,7.443911534483058e-12,
                               8.031960755589202e-13,2.6575301338450717e-13,5.004072698835933e-14,
                           1.187407081079713e-14,8.198763178883733e-15,7.350615263826794e-15,2.0355549961366507e-14])
# Degree of Polynomial
Nrange = list(range(2,15))
lp_degree=float("inf")
refinement=0
error1=[] 
for n in Nrange:
    if n%1==0:print(n)
    erro1 = err_t(int(n),lp_degree,newt_poly,newt_poly.compute_gradients_at,mesh_path,refinement)
    error1.append(erro1)

plt.semilogy(Nrange, error1, '-or')
plt.semilogy(Nrange, dune2_14_N_4024, '-ob')
plt.xlabel("Degree of Polynomial",fontsize=13)
plt.ylabel("Relative Error",fontsize=13)
plt.legend(['HOSQ+GPLS','DCG'],prop={'size': 13},loc='upper center')
plt.xticks(np.arange(min(Nrange), max(Nrange)+1, 1.0))
plt.ylim([2.758195177427762e-18,3.9514540203871754e-04])
plt.grid()
plt.savefig("../images/dune_vs_ellipsoidHOSQ+GPLS_4024.pdf")